In [3]:
import pandas as pd
from os.path import join

In [6]:
configs = {
    'fold1': [
        'competitions/2020/melanoma-classification/configs/efficientnet/best-ns-colorjitter',
        'competitions/2020/melanoma-classification/configs/efficientnet/best-ns-colorjitter-effb6',
        'competitions/2020/melanoma-classification/configs/efficientnet/best-ns-colorjitter-effb7'
    ],
    'fold2': [
        'competitions/2020/melanoma-classification/configs/efficientnet-cv/best-ns-colorjitter-fold2',
        'competitions/2020/melanoma-classification/configs/efficientnet-cv/best-ns-colorjitter-effb6-fold2',
        'competitions/2020/melanoma-classification/configs/efficientnet-cv/best-ns-colorjitter-effb7-fold2'
    ],
    'fold3': [
        'competitions/2020/melanoma-classification/configs/efficientnet-cv/best-ns-colorjitter-fold3',
        'competitions/2020/melanoma-classification/configs/efficientnet-cv/best-ns-colorjitter-effb6-fold3',
        'competitions/2020/melanoma-classification/configs/efficientnet-cv/best-ns-colorjitter-effb7-fold3'
    ],
    'fold4': [
        'competitions/2020/melanoma-classification/configs/efficientnet-cv/best-ns-colorjitter-fold4',
        'competitions/2020/melanoma-classification/configs/efficientnet-cv/best-ns-colorjitter-effb6-fold4',
        'competitions/2020/melanoma-classification/configs/efficientnet-cv/best-ns-colorjitter-effb7-fold4'
    ],
    'fold5': [
        'competitions/2020/melanoma-classification/configs/efficientnet-cv/best-ns-colorjitter-fold5',
        'competitions/2020/melanoma-classification/configs/efficientnet-cv/best-ns-colorjitter-effb6-fold5',
        'competitions/2020/melanoma-classification/configs/efficientnet-cv/best-ns-colorjitter-effb7-fold5'
    ]
}

In [7]:
all_predictions = []

for index, (fold, fold_configs) in enumerate(configs.items()):
    fold_predictions = None
    
    for config_index, config in enumerate(fold_configs):
        pred_path = join('/output', config, 'logs/evaluation/test.csv')
        config_predictions = pd.read_csv(pred_path)
        config_predictions = config_predictions.rename(columns={'target': f'target{config_index+1}'})
        if fold_predictions is None:
            fold_predictions = config_predictions
        else:
            fold_predictions = pd.merge(fold_predictions, config_predictions, on='image_name')
    
    all_predictions.append(fold_predictions)
#     pred_path = join('/output', config, 'logs/evaluation/test.csv')
#     fold_predictions = pd.read_csv(pred_path)
#     fold_predictions = fold_predictions.rename(columns={'target': f'target{index+1}'})
#     if all_predictions is None:
#         all_predictions = fold_predictions
#     else:
#         all_predictions = pd.merge(all_predictions, fold_predictions, on='image_name')

In [8]:
all_predictions[0].head()

,image_name,target1,target2,target3
0,ISIC_0052060,0.018915,0.021542,0.107061
1,ISIC_0052349,0.003282,0.007053,0.023991
2,ISIC_0058510,0.007697,0.007457,0.026057
3,ISIC_0073313,0.004560,0.001320,0.036480
4,ISIC_0073502,0.051950,0.025945,0.128810


In [9]:
all_predictions[1].head()

,image_name,target1,target2,target3
0,ISIC_0052060,0.034260,0.003016,0.015133
1,ISIC_0052349,0.003239,0.000839,0.001366
2,ISIC_0058510,0.008867,0.024391,0.006551
3,ISIC_0073313,0.002018,0.001111,0.001509
4,ISIC_0073502,0.074716,0.000721,0.015911


In [10]:
target_columns = [f'target{index+1}' for index in range(3)]

In [15]:
for index in range(len(all_predictions)):
    fold_predictions = all_predictions[index]
    fold_predictions[f'fold{index+1}'] = fold_predictions[target_columns].mean(axis=1)
    fold_predictions = fold_predictions.drop(columns=target_columns)
    all_predictions[index] = fold_predictions

In [18]:
fold_columns = [f'fold{index+1}' for index in range(5)]

In [19]:
final_predictions = all_predictions[0]

for index in range(1, len(all_predictions)):
    final_predictions = pd.merge(final_predictions, all_predictions[index], on='image_name')

In [21]:
final_predictions['target'] = final_predictions[fold_columns].mean(axis=1)

In [23]:
final_predictions = final_predictions.drop(columns=fold_columns)

In [24]:
final_predictions

,image_name,target
0,ISIC_0052060,0.021670
1,ISIC_0052349,0.005016
2,ISIC_0058510,0.009002
3,ISIC_0073313,0.004590
4,ISIC_0073502,0.027846
...,...,...
10977,ISIC_9992485,0.056644
10978,ISIC_9996992,0.316587
10979,ISIC_9997917,0.357863
10980,ISIC_9998234,0.024920


In [ ]:
final_predictions.to_csv('submissions/ensembles/effnet-b5-b6-b7-v2.csv', index=False)